In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sb

##### Data Preparation
* Dataset are spread across 5 csv files

In [76]:
# df_90to99 = pd.read_csv('../DataSets/SingaporeResaleFlatData/resale-flat-prices-based-on-approval-date-1990-1999.csv')
# df_00to12 = pd.read_csv('../DataSets/SingaporeResaleFlatData/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
df_12to14 = pd.read_csv('../DataSets/SingaporeResaleFlatData/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
df_1516 = pd.read_csv('../DataSets/SingaporeResaleFlatData/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
df_17 = pd.read_csv('../DataSets/SingaporeResaleFlatData/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')


In [77]:
df_test = pd.concat([df_12to14,df_1516,df_17], axis = 0)

In [81]:
df_test[df_test['remaining_lease'].notnull()]['remaining_lease']

0                         70
1                         65
2                         64
3                         63
4                         64
                 ...        
130399    64 years 07 months
130400              65 years
130401    64 years 11 months
130402    64 years 09 months
130403    64 years 09 months
Name: remaining_lease, Length: 167557, dtype: object

In [ ]:
[]

In [63]:
# df_00to12
# df_90to99
df = pd.concat([df_12to14,df_1516,df_17], axis = 0)

In [64]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN


In [65]:
df.shape

(219760, 11)

In [29]:
df.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
remaining_lease         object
dtype: object

In [66]:
df['month'] = pd.to_datetime(df['month'])

In [67]:
df.isnull().sum()

month                      0
town                       0
flat_type                  0
block                      0
street_name                0
storey_range               0
floor_area_sqm             0
flat_model                 0
lease_commence_date        0
resale_price               0
remaining_lease        52203
dtype: int64

##### HDB Flats in Singapore have a lease of 99 years
Therefore, remaining_lease = 99 - (year - lease_commence_date)

In [68]:
df.rename(columns = {'month':'date'}, inplace = True)

In [69]:
df['year'] = [i.year for i in df['date']]
df['month'] = [i.month for i in df['date']]

In [70]:
df.drop('date', axis = 1, inplace = True)
df.reset_index(inplace = True, drop = True)

In [71]:
df.head()

,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,year,month
0,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,2012,3
1,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN,2012,3
2,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN,2012,3
3,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN,2012,3
4,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN,2012,3


##### Column 'remaining_lease' contains some data in format : '61 years 04 months'. Need to standardize to just year

In [72]:
mask = (df['remaining_lease'].notnull())
for i in df.loc[mask, 'remaining_lease']:
    if not isinstance(i, int):
        df.loc[mask, 'remaining_lease'] = int(i[:2])

In [73]:
99 - (df.loc[0]['year'] - df.loc[0]['lease_commence_date'])

73

In [94]:
for index in df[df['remaining_lease'].isna()].index:
    df.loc[index, 'remaining_lease'] = 99 - (int(df.loc[index]['year'] - df.loc[index]['lease_commence_date']))

86

In [81]:
for i in df[df['remaining_lease'].isnull()]['remaining_lease']

,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,year,month
0,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,1990,1
1,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,1990,1
2,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,1990,1
3,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,1990,1
4,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,1990,1
...,...,...,...,...,...,...,...,...,...,...,...,...
52198,YISHUN,5 ROOM,816,YISHUN ST 81,10 TO 12,122.0,Improved,1988,580000.0,NaN,2014,12
52199,YISHUN,EXECUTIVE,325,YISHUN CTRL,10 TO 12,146.0,Maisonette,1988,540000.0,NaN,2014,12
52200,YISHUN,EXECUTIVE,618,YISHUN RING RD,07 TO 09,164.0,Apartment,1992,738000.0,NaN,2014,12
52201,YISHUN,EXECUTIVE,277,YISHUN ST 22,07 TO 09,152.0,Maisonette,1985,592000.0,NaN,2014,12
